# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries


import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('DisasterResponse', con=engine)
X =  df.message.values
Y =  df.iloc[:,5:]



### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
   
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
  
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:

pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])
pipe.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fa556cfb268>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

pipe.fit(X_train, Y_train)




Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def eval_metrics(array1, array2, col_names):
    metrics = []
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        print(col_names[i])
        print(classification_report(array1[:,i],array2[:,i]))

In [8]:
Y_test_pred = pipe.predict(X_test)
col_names = list(Y.columns.values)
eval_metrics(np.array(Y_test), Y_test_pred, col_names)


request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5472
          1       0.84      0.41      0.55      1125

avg / total       0.88      0.89      0.87      6597

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6561
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6597

aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3865
          1       0.75      0.51      0.61      2732

avg / total       0.73      0.73      0.71      6597

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0.96      6097
          1       0.63      0.06      0.11       500

avg / total       0.91      0.93      0.90      6597

medical_products
             precision    recall  f1-score   support

          0       0.95      1.

In [ ]:
Y_train_pred = pipe.predict(X_train)
col_names = list(Y.columns.values)
eval_metrics(np.array(Y_train), Y_train_pred, col_names)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

parameters = {'clf__estimator__min_samples_split':[2, 5, 10]}


cv_fit = GridSearchCV(pipe, param_grid = parameters,verbose = 1)

# Find best parameters
np.random.seed(74)
cv = cv_fit.fit(X_train, Y_train)


##### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
tuned_pred_test = cv_fit.predict(X_test)

eval_cv_test = eval_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(eval_cv_test)

eval_cv_test.describe()

##### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
parameters = {'clf__estimator__n_estimators': [50, 100],
                  'clf__estimator__min_samples_split': [2, 3, 4],
                  'clf__estimator__criterion': ['entropy', 'gini']
             }
cv_fit2 = GridSearchCV(pipe, param_grid = parameters, verbose = 3)

tuned_model = cv_fit2.fit(X_train,Y_train)
tuned_pred_tes2 = tuned_model.predict(X_test)
eval_cv_tes2 = eval_metrics(np.array(Y_test), tuned_pred_tes2,col_names)
print(eval_cv_tes2)

In [ ]:
#USing logistic regression

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pca= PCA()
logistic = LogisticRegression(max_iter=10000, tol=0.1)
pipe2 = ([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('pca', pca), 
    ('logistic', logistic)
])

search = GridSearchCV(pipe2, param_grid, n_jobs=-1)
model1 = search.fit(X_train,Y_train)
tuned_mod1 = model1.predict(X_test)
eval_cv_tes3 = eval_metrics(np.array(Y_test),tuned_mod1,col_names)
print(eval_cv_tes2)

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.